# Medical Chatbot

### Using Llama 3 and RAG for a medical chatbot

In [23]:
!pip install huggingfacehub
!pip install transformers
!pip install accelerate
!pip install torch
!pip install langchain
!pip install pypdf

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


ERROR: Could not find a version that satisfies the requirement huggingfacehub (from versions: none)
ERROR: No matching distribution found for huggingfacehub


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/290.4 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/290.4 kB 660.6 kB/s eta 0:00:01
   ---------------- ----------------------- 122.9/290.4 kB 1.8 MB/s eta 0:00:01
   ----------------------------------- ---- 256.0/290.4 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 290.4/290.4 kB 1.8 MB/s eta 0:00:00


In [32]:
import transformers
import torch
import requests
import huggingface_hub
import os
import bs4

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser # Can be customized

from dotenv import load_dotenv

from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader # All the techniques to load all the documents 
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

In [29]:
# Load the PDF using Langchain

loader= PyPDFLoader(r"C:\Users\ganad\Desktop\Github Learning Freecodecamp\Llama3_Medical_Chatbot\documents\125514s096lbl.pdf")
document = loader.load()
pages = loader.load_and_split()

In [28]:
pages[0]

Document(page_content='HIGHLIGHTS OF PRESCRIBING INFORMATION \nThese highlights do not include a ll the information needed to u se \nKEYTRUDA safely and effectively. See full prescribing information for KEYTRUDA. \nKEYTRUDA\n® (pembrolizumab) injection, for intravenous use \nInitial U.S. Approval: 2014 \n--------------------------- RECENT MAJOR CHANGES ------------- -------------- \nIndications and Usage (1) 03/2021 \nDosage and Administration (2) 03/2021 \nWarnings and Precautions (5) 11/2020 \n----------------------------INDICATIONS AND USAGE------------ ---------------- \nKEYTRUDA is a programmed death receptor-1 (PD-1)-blocking \nantibody indicated:  \nMelanoma \n\uf0b7 for the treatment of patients with unresectable or metastatic \nmelanoma. (1.1) \n\uf0b7 for the adjuvant treatment of patients with melanoma with \ninvolvement of lymph node(s) following complete resection. (1.1)\n\t\nNon-Small Cell Lung Cancer (NSCLC)\n\t\n\uf0b7 in combination with pemetrexed and platinum chemoth

In [33]:
document

[Document(page_content=' \n   \n \n \n   \n \n   \n  \n  \n  \n   \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n  \n  \n \n  \n \n  \n \n  \n \n \n \n \n \n \n \n  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n  \n \n \n \n  HIGHLIGHTS OF PRESCRIBING INFORMATION \nThese highlights do not include a ll the information needed to u se \nKEYTRUDA safely and effectively. See full prescribing information for KEYTRUDA. \nKEYTRUDA\n® (pembrolizumab) injection, for intravenous use \nInitial U.S. Approval: 2014 \n--------------------------- RECENT MAJOR CHANGES ------------- -------------- \nIndications and Usage (1) 03/2021 \nDosage and Administration (2) 03/2021 \nWarnings and Precautions (5) 11/2020 \n----------------------------INDICATIONS AND USAGE------------ ---------------- \nKEYTRUDA is a programmed death receptor-1 (PD-1)-blocking \nantibody indicated:  \nMelanoma \n\uf0b7 for the 

In [31]:
load_dotenv()

os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [34]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(document)
documents[:5]

[Document(page_content='HIGHLIGHTS OF PRESCRIBING INFORMATION \nThese highlights do not include a ll the information needed to u se \nKEYTRUDA safely and effectively. See full prescribing information for KEYTRUDA. \nKEYTRUDA\n® (pembrolizumab) injection, for intravenous use \nInitial U.S. Approval: 2014 \n--------------------------- RECENT MAJOR CHANGES ------------- -------------- \nIndications and Usage (1) 03/2021 \nDosage and Administration (2) 03/2021 \nWarnings and Precautions (5) 11/2020 \n----------------------------INDICATIONS AND USAGE------------ ---------------- \nKEYTRUDA is a programmed death receptor-1 (PD-1)-blocking \nantibody indicated:  \nMelanoma \n\uf0b7 for the treatment of patients with unresectable or metastatic \nmelanoma. (1.1) \n\uf0b7 for the adjuvant treatment of patients with melanoma with', metadata={'source': 'C:\\Users\\ganad\\Desktop\\Github Learning Freecodecamp\\Llama3_Medical_Chatbot\\documents\\125514s096lbl.pdf', 'page': 0}),
 Document(page_conten

In [35]:
documents 

[Document(page_content='HIGHLIGHTS OF PRESCRIBING INFORMATION \nThese highlights do not include a ll the information needed to u se \nKEYTRUDA safely and effectively. See full prescribing information for KEYTRUDA. \nKEYTRUDA\n® (pembrolizumab) injection, for intravenous use \nInitial U.S. Approval: 2014 \n--------------------------- RECENT MAJOR CHANGES ------------- -------------- \nIndications and Usage (1) 03/2021 \nDosage and Administration (2) 03/2021 \nWarnings and Precautions (5) 11/2020 \n----------------------------INDICATIONS AND USAGE------------ ---------------- \nKEYTRUDA is a programmed death receptor-1 (PD-1)-blocking \nantibody indicated:  \nMelanoma \n\uf0b7 for the treatment of patients with unresectable or metastatic \nmelanoma. (1.1) \n\uf0b7 for the adjuvant treatment of patients with melanoma with', metadata={'source': 'C:\\Users\\ganad\\Desktop\\Github Learning Freecodecamp\\Llama3_Medical_Chatbot\\documents\\125514s096lbl.pdf', 'page': 0}),
 Document(page_conten

In [39]:
!pip install chromadb

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 0.0/67.3 kB ? eta -:--:--
     ---------------------------------------- 67.3/67.3 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/525.5 kB ? eta -:--:--
   ----- ---------------------------------- 71.7/525.5 kB 3.8 MB/s eta 0:00:01
   ----------------- ---------------------- 235.5/525.5 kB 2.9 MB/s eta 0:00:01
   ---------------------------- ----------- 368.6/525.5 kB 2.9 MB/s eta 0:00:01
   ---------------------------------------- 525.5/525.5 kB 3.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/150.5 k

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.9.0 which is incompatible.


In [40]:
!pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/14.5 MB 1.7 MB/s eta 0:00:09
    --------------------------------------- 0.2/14.5 MB 1.5 MB/s eta 0:00:10
    --------------------------------------- 0.3/14.5 MB 1.4 MB/s eta 0:00:10
    --------------------------------------- 0.4/14.5 MB 1.6 MB/s eta 0:00:09
   - -------------------------------------- 0.5/14.5 MB 1.8 MB/s eta 0:00:08
   - -------------------------------------- 0.6/14.5 MB 1.7 MB/s eta 0:00:09
   - -------------------------------------- 0.6/14.5 MB 1.8 MB/s eta 0:00:08
   -- ------------------------------------- 0.7/14.5 MB 1.8 MB/s eta 0:00:08
   -- ------------------------------------- 0.8/14.5 MB 1.9 MB/s eta 0:00:08
   -- ------------------------------------- 1.0/14.5 MB 1.9 MB/s eta 0:00:08
   -- -----------

In [42]:
# Vector Embeddings and Vector Store
# https://python.langchain.com/docs/integrations/text_embedding/ollama/ 
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(documents, OllamaEmbeddings())

'''


from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

# Initialize OllamaEmbeddings with the specific model name
embeddings = OllamaEmbeddings(model_name='llama3')
db = FAISS.from_documents(documents, embeddings)

'''

ValueError: Error raised by inference API HTTP code: 404, {"error":"model 'llama2' not found, try pulling it first"}

In [ ]:
# Vector database
query = ""
result = db.similarity_search(query, top_k=5)

### References:
- https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf/
- https://python.langchain.com/docs/integrations/text_embedding/ollama/